# Limpieza y transformacion de los archivos de datos

In [17]:
import pandas as pd
import numpy as np
import locale
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')

'Spanish_Spain.1252'

In [19]:
df_bancos = pd.read_csv("bank-additional.csv")
hojas_excel = pd.read_excel("customer-details.xlsx", sheet_name = None)
df_clientes = pd.concat(hojas_excel.values(), ignore_index = True)

## Limpieza

In [20]:
for col in ['cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']:
    df_bancos[col] = pd.to_numeric(df_bancos[col].str.replace(',', '.'), errors='coerce')

In [21]:
df_bancos['date'] = pd.to_datetime(df_bancos['date'], format='%d-%B-%Y', errors='coerce')

In [22]:
df_bancos = df_bancos.rename(columns = {'id_':'ID'})

In [23]:
df_clientes['Total_kids'] = df_clientes['Kidhome'] + df_clientes['Teenhome']

In [24]:
#Calcular el numero de dias que ha sido cliente cada uno
today = pd.to_datetime('2023-12-31')  
df_clientes['Customer_seniority'] = (today - df_clientes['Dt_Customer']).dt.days

In [25]:
df_bancos.isnull().sum()

Unnamed: 0           0
age               5120
job                345
marital             85
education         1807
default           8981
housing           1026
loan              1026
contact              0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx     471
cons.conf.idx        0
euribor3m         9256
nr.employed          0
y                    0
date               248
latitude             0
longitude            0
ID                   0
dtype: int64

In [26]:
columns = df_bancos.columns[df_bancos.isnull().any()].tolist() #Convertir a lista las columnas que tienen valores nulos para poder usar el for
for columna in columns:
    print(f"Valores totales en {columna}:")
    print(df_bancos[columna].count())
    print(f"Valores nulos en {columna}:")
    print(df_bancos[columna].isnull().sum())
    print(f"Porcentaje de valores nulos en {columna}: {df_bancos[columna].isnull().sum() / len(df_bancos) * 100:.2f}%")
    print("\n")

Valores totales en age:
37880
Valores nulos en age:
5120
Porcentaje de valores nulos en age: 11.91%


Valores totales en job:
42655
Valores nulos en job:
345
Porcentaje de valores nulos en job: 0.80%


Valores totales en marital:
42915
Valores nulos en marital:
85
Porcentaje de valores nulos en marital: 0.20%


Valores totales en education:
41193
Valores nulos en education:
1807
Porcentaje de valores nulos en education: 4.20%


Valores totales en default:
34019
Valores nulos en default:
8981
Porcentaje de valores nulos en default: 20.89%


Valores totales en housing:
41974
Valores nulos en housing:
1026
Porcentaje de valores nulos en housing: 2.39%


Valores totales en loan:
41974
Valores nulos en loan:
1026
Porcentaje de valores nulos en loan: 2.39%


Valores totales en cons.price.idx:
42529
Valores nulos en cons.price.idx:
471
Porcentaje de valores nulos en cons.price.idx: 1.10%


Valores totales en euribor3m:
33744
Valores nulos en euribor3m:
9256
Porcentaje de valores nulos en euri

In [27]:
# 1. Calculamos los cuartiles de 'age'
Q1 = df_bancos['age'].quantile(0.25)  # primer cuartil
Q3 = df_bancos['age'].quantile(0.75)  # tercer cuartil

# 2. Calculamos el rango intercuartílico (IQR)
IQR = Q3 - Q1

# 3. Definimos límites inferior y superior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
# 4. Filtramos el DataFrame para mantener sólo valores dentro de los límites
df_bancos = df_bancos[
    (df_bancos['age'] >= limite_inferior) & 
    (df_bancos['age'] <= limite_superior)
]

In [28]:
bins = [17, 25, 35, 50, 65, df_bancos['age'].max()]
labels = ['18-25', '26-35', '36-50', '51-65', '66+']

# Crear la nueva columna 'grupo_edad' utilizando pd.cut
grupo_edad = pd.cut(df_bancos['age'], bins=bins, labels=labels)

# Insertamos justo después de la columna 'age'
pos = df_bancos.columns.get_loc('age') + 1
df_bancos.insert(pos, 'grupo_edad', grupo_edad)

In [ ]:
# Columnas numéricas (rellenar con mediana)
for col in ['age', 'cons.price.idx']:
    median_value = df_bancos[col].median()
    df_bancos[col] = df_bancos[col].fillna(median_value)

# Columnas categóricas (rellenamos con 'UNKNOWN')
for col in ['job', 'marital', 'education']: #, 'housing', 'loan']:
    df_bancos[col] = df_bancos[col].fillna('UNKNOWN')

# Columnas con muchos nulos y que consideramos que no son utiles en nuestro analisis (eliminar completamente)
df_bancos = df_bancos.drop(columns=['euribor3m'])

In [30]:
df_union = pd.merge(df_bancos, df_clientes, on='ID', how='inner')

In [31]:
#Poner el nombre de las columnas en mayúsculas
new_columns = []
for col in df_union.columns:
    new_columns.append(col.upper())

df_union.columns = new_columns
df_union

,UNNAMED: 0_X,AGE,GRUPO_EDAD,JOB,MARITAL,EDUCATION,DEFAULT,HOUSING,LOAN,CONTACT,...,LONGITUDE,ID,UNNAMED: 0_Y,INCOME,KIDHOME,TEENHOME,DT_CUSTOMER,NUMWEBVISITSMONTH,TOTAL_KIDS,CUSTOMER_SENIORITY
0,1,57.0,51-65,services,MARRIED,high.school,NaN,0.0,0.0,telephone,...,-83.923,e9d37224-cb6f-4942-98d7-46672963d097,1,85477,1,1,2012-12-30,7,2,4018
1,2,37.0,36-50,services,MARRIED,high.school,0.0,1.0,0.0,telephone,...,-94.847,3f9f49b5-e410-4948-bf6e-f9244f04918b,2,147233,1,1,2012-02-02,5,2,4350
2,3,40.0,36-50,admin.,MARRIED,basic.6y,0.0,0.0,0.0,telephone,...,-70.308,9991fafb-4447-451a-8be2-b0df6098d13e,3,121393,1,2,2012-12-21,29,3,4027
3,4,56.0,51-65,services,MARRIED,high.school,0.0,0.0,1.0,telephone,...,-104.463,eca60b76-70b6-4077-80ba-bc52e8ebb0eb,4,63164,1,2,2012-06-20,20,3,4211
4,5,45.0,36-50,services,MARRIED,basic.9y,NaN,0.0,0.0,telephone,...,-101.643,d63ede72-0b6d-45b1-8872-385ac6897f65,5,143854,0,1,2012-01-24,26,1,4359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37434,28721,36.0,36-50,blue-collar,MARRIED,UNKNOWN,0.0,1.0,1.0,cellular,...,-93.780,649cf395-b67a-416c-b9ae-3eaf6d3661c5,14077,129333,0,2,2014-04-10,15,2,3552
37435,37852,26.0,26-35,student,SINGLE,high.school,0.0,0.0,0.0,cellular,...,-86.270,12d4e85c-39d9-4193-a27d-a58e7af15a43,14078,169378,1,2,2014-12-31,30,3,3287
37436,1120,34.0,26-35,blue-collar,SINGLE,basic.9y,0.0,0.0,0.0,telephone,...,-95.459,0490dbb9-e21d-4b59-a402-93756e8f17da,14080,159144,1,0,2014-10-14,1,1,3365
37437,3797,30.0,26-35,admin.,MARRIED,university.degree,0.0,1.0,0.0,telephone,...,-74.361,d9f2c31c-7623-44df-9240-b4514bf21abd,14081,51496,0,0,2014-09-18,12,0,3391


In [32]:
df_union.to_csv("dataset_limpio.csv", index = False)